In [1]:
import pandas as pd
import numpy as np
import urllib.request
import json
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, Point
from shapely import wkt
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.express as px
import warnings
from sklearn.neighbors import KNeighborsClassifier
import os
from scipy.stats import f_oneway
from pathlib import Path

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', '{:.3f}'.format)
warnings.filterwarnings('ignore')

In [3]:
raw = "C:\\Users\\taavi\\Desktop\\BPHIL\\Raw data\\"
clean = "C:\\Users\\taavi\\Desktop\\BPHIL\\Clean data\\"

### Filtering logic for within Pittsburgh boundary

In [4]:
# https://pghgishub-pittsburghpa.opendata.arcgis.com/datasets/pittsburghpa::pittsburgh-boundary/explore?location=40.430398%2C-79.979853%2C11.85
geojson = gpd.read_file(raw + 'Pittsburgh_Boundary.geojson')

boundary = geojson.geometry.iloc[2]
# this is the perimeter of the city

polygons = [geom for geom in boundary.geoms]
    
def filterData(data, polygons):
    filteredData = []
    for index, row in data.iterrows():
        dataPoint = Point(row['lng'], row['lat'])
        for polygon in polygons:
            if dataPoint.within(polygon):
                filteredData.append(row)
                break
    return pd.DataFrame(filteredData)

### Neighborhoods

In [60]:
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=668d7238-cfd2-492e-b397-51a6e74182ff&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break
        
nbrhds = pd.concat(dfs.values(), ignore_index = True)

In [61]:
nbrhds.to_csv(raw + 'raw_neighborhoods.csv', index = False)

In [62]:
nbrhds = pd.read_csv(raw + 'raw_neighborhoods.csv')

In [63]:
nbrhds = (
    nbrhds
    .assign(
        lat = nbrhds['intptlat10'].apply(lambda x: float(x[1:] if x != ' ' else np.nan)),
        lng = nbrhds['intptlon10'].apply(lambda x: float(x) if x != ' ' else np.nan)
    )
)

In [64]:
nbrhds.to_csv(clean + 'clean_nbrhds.csv', index = False)

In [5]:
nbrhds = pd.read_csv(clean + 'clean_nbrhds.csv')

### Parcels

In [66]:
# https://data.wprdc.org/dataset/parcel-centroids-in-allegheny-county-with-geographic-identifiers/resource/adf1fd38-c374-4c4e-9094-5e53bd12419f
# - 2010 census boundaries and redistricting prior to 2020
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=adf1fd38-c374-4c4e-9094-5e53bd12419f&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break
        
parcels = pd.concat(dfs.values(), ignore_index = True)

KeyboardInterrupt: 

In [ ]:
parcels.shape

(585387, 23)

In [ ]:
parcels.head(1)

In [ ]:
parcels.to_csv(raw + 'raw_parcels.csv', index = False)

In [ ]:
parcels = pd.read_csv(raw + 'raw_parcels.csv')

In [ ]:
# neighborhood
parcels = parcels.loc[parcels['geo_name_n'] != ' ']
parcels['geo_name_n'] = np.where(parcels['geo_name_n'] != 'Mount Oliver Borough', parcels['geo_name_n'], 'Mt. Oliver')

# isolate 'parcelID', 'lat', 'lng', 'nbrhd', 'tract'
parcels = parcels.loc[:, ['PIN', 'Latitude', 'Longitude', 'geo_name_n', 'TRACTCE10', 'BLOCKCE10']] # could include GEOID10 for census bureau statistics mapping

# rename columns
parcels.columns = ['parcelID', 'lat', 'lng', 'nbrhd', 'tract', 'block'] 

# drop nulls - string 'nan' in this case
#parcels = parcels.drop(index = [583953, 583954])
exclude = [' ', 'COMMON GROUND', 'Westmoreland County', 'Washington County', 'Butler County', 'Not Assessed', 'Beaver County']
parcels = parcels.loc[parcels['parcelID'].isin(exclude) == False]

parcels = parcels.drop_duplicates(subset = 'parcelID')

# filter for just Pittsburgh
parcels = filterData(parcels, polygons)

parcels = parcels.reset_index(drop = True)

In [ ]:
parcels.head(1)

In [ ]:
parcels.shape[0]

In [ ]:
parcels.to_csv(clean + 'clean_parcels.csv', index = False)

In [6]:
parcels = pd.read_csv(clean + 'clean_parcels.csv')

### Violations - new

In [ ]:
# https://data.wprdc.org/dataset/pittsburgh-pli-violations-report/resource/70c06278-92c5-4040-ab28-17671866f81c
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=70c06278-92c5-4040-ab28-17671866f81c&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

viols = pd.concat(dfs.values(), ignore_index = True)

In [ ]:
viols.head(1)

In [ ]:
viols.to_csv(raw + 'raw_viols.csv', index = False)

In [ ]:
viols = pd.read_csv(raw + 'raw_viols.csv')

In [ ]:
# merge in lat and lng, and also filter for within Pittsburgh
viols = viols.merge(right = parcels, left_on = 'parcel_id', right_on = 'parcelID', how = 'inner')

In [ ]:
viols.head(1)

In [ ]:
# isolate columns
viols = viols.loc[:, ['casefile_number', 'parcel_id', 'investigation_date', 'violation_code_section', 'investigation_outcome', 'lat', 'lng']]

# rename columns
viols.columns = ['violID', 'parcelID', 'date', 'code', 'outcome', 'lat', 'lng']

# drop nulls
viols = viols.dropna()

# drop duplicates
#viols = viols.drop_duplicates()

In [ ]:
viols.head(1)

In [ ]:
viols.to_csv(clean + 'clean_viols.csv', index = False)

### Violations - old

In [ ]:
# https://data.wprdc.org/dataset/pittsburgh-pli-violations-report/resource/4e5374be-1a88-47f7-afee-6a79317019b4
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=4e5374be-1a88-47f7-afee-6a79317019b4&limit=32000'  
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

violsOld = pd.concat(dfs.values(), ignore_index = True)

In [ ]:
violsOld.to_csv(raw + 'raw_viols_old.csv', index = False)

In [ ]:
violsOld.head(1)

In [ ]:
violsOld = pd.read_csv(raw + 'raw_viols_old.csv')

In [ ]:
violsOld = violsOld.loc[violsOld['INSPECTION_RESULT'] == 'Violations Found']

violsOld = violsOld[['CASE_NUMBER', 'PARCEL', 'INSPECTION_DATE', 'VIOLATION', 'INSPECTION_RESULT', 'Y', 'X']]

violsOld.columns = ['violID', 'parcelID', 'date', 'code', 'outcome', 'lat', 'lng']

violsOld = violsOld.dropna()

In [ ]:
violsOld.head(1)

In [ ]:
violsOld.to_csv(clean + 'clean_viols_old.csv', index = False)

In [ ]:
violsOld = pd.read_csv(clean + 'clean_viols_old.csv')

### Crimes - 2016-2023

In [ ]:
# https://data.wprdc.org/dataset/uniform-crime-reporting-data/resource/044f2016-1dfd-4ab0-bc1e-065da05fca2e
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=044f2016-1dfd-4ab0-bc1e-065da05fca2e&limit=32000'  
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

crimes1 = pd.concat(dfs.values(), ignore_index = True)

In [ ]:
crimes1.head(1)

In [ ]:
crimes1.to_csv(raw + 'raw_crimes_old.csv', index = False)

In [ ]:
crimes1 = pd.read_csv(raw + 'raw_crimes_old.csv')

In [ ]:
part1_offenses = [
    'HOMICIDE', 'RAPE', 'ROBBERY', 'AGGRAVATED ASSAULT', 'BURGLARY', 'THEFT', 'ARSON', 'TRAFFICKING', 'ASSAULT', 'MURDER'
]

part2_offenses = [
    'OTHER ASSAULTS', 'FORGERY AND COUNTERFEITING', 'FRAUD', 'EMBEZZLEMENT', 'STOLEN PROPERTY', 'VANDALISM', 'WEAPONS', 'FIREARM', 'PROSTITUTION',
    'SEX OFFENSES','DRUG ABUSE VIOLATIONS', 'GAMBLING', 'OFFENSES AGAINST THE FAMILY AND CHILDREN', 'DRIVING UNDER THE INFLUENCE', 'LIQUOR',
    'DRUNKENNESS', 'DISORDERLY CONDUCT', 'VAGRANCY', 'ALL OTHER OFFENSES', 'SUSPICION', 'CURFEW', 'LOITERING'
]

In [ ]:
def classify_ucr(desc):
    if pd.isna(desc):
        return 'Unknown'
    desc_upper = desc.upper()
    for offense in part1_offenses:
        if offense in desc_upper:
            return 'High'
    for offense in part2_offenses:
        if offense in desc_upper:
            return 'Low'
    return 'Low'

In [ ]:
crimes1['severity'] = crimes1['OFFENSES'].apply(classify_ucr)

In [ ]:
crimes1 = crimes1[['OFFENSES', 'severity', 'INCIDENTTIME', 'INCIDENTLOCATION', 'INCIDENTNEIGHBORHOOD', 'INCIDENTZONE', 'INCIDENTTRACT', 'Y', 'X']]

crimes1.columns = ['crime', 'severity', 'date', 'address', 'nbrhd', 'zone', 'tract', 'lat', 'lng']

crimes1 = filterData(crimes1, polygons).reset_index(drop = True)

# crimes1 = crimes1.drop_duplicates(subset = ['crime', 'date', 'address'])
# not dropping duplicates because data disclaimer says location is block-address rather than street-address to protect identity
# so I can't say that dupe addresses or even dupe coordinates are also dupe incidents
# https://www.pittsburghpa.gov/files/assets/city/v/1/public-safety/documents/police/2025_pittsburgh_monthly_incident_dataset_disclaimer.pdf

# crimes1['date'] = pd.to_datetime(crimes1['date'])

In [ ]:
(crimes1[['lat', 'lng']] == 0).sum()

lat    0
lng    0
dtype: int64

In [ ]:
# toMerge = nbrhds[['hood', 'lat', 'lng']]
# toMerge.columns = ['nbrhd', 'lat_nbrhd', 'lng_nbrhd']
# crimes1 = (
#     crimes1
#     .merge(right = toMerge, on = 'nbrhd', how = 'left')
#     .assign(
#         lat = lambda x: np.where((x['lat'].notnull()) & (x['lat'] != 0), x['lat'], x['lat_nbrhd']),
#         lng = lambda x: np.where((x['lng'].notnull()) & (x['lng'] != 0), x['lng'], x['lng_nbrhd'])
#     )
#     .dropna(subset = ['lat', 'lng'])
# )

In [ ]:
crimes1.to_csv(clean + 'clean_crimes_old.csv', index = False)

In [ ]:
crimes1 = pd.read_csv(clean + 'clean_crimes_old.csv')

### Crimes - 2024-2025

In [ ]:
# https://data.wprdc.org/dataset/monthly-criminal-activity-dashboard/resource/bd41992a-987a-4cca-8798-fbe1cd946b07
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=bd41992a-987a-4cca-8798-fbe1cd946b07&limit=32000'  
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

crimes2 = pd.concat(dfs.values(), ignore_index = True)

In [ ]:
crimes2.to_csv(raw + 'raw_crimes_new.csv', index = False)

In [67]:
crimes2 = pd.read_csv(raw + 'raw_crimes_new.csv')

In [68]:
crimes2['severity'] = crimes2['Violation'].apply(classify_ucr)

crimes2 = (
    crimes2
    .assign(
        sev_code = np.where(crimes2['severity'] == 'High', 1, 0)
    )
    .assign(
        sev_code = lambda x: x.groupby('Report_Number')['sev_code'].transform('max')
    )
    .assign(
        severity = lambda x: np.where(x['sev_code'] == 1, 'High', 'Low')
    )
    .drop_duplicates(subset = 'Report_Number')
)

In [70]:
crimes2 = crimes2[['Violation', 'severity', 'ReportedDate', 'Block_Address', 'Neighborhood', 'Zone', 'Tract', 'YCOORD', 'XCOORD']]

crimes2.columns = ['crime', 'severity', 'date', 'address', 'nbrhd', 'zone', 'tract', 'lat', 'lng']

crimes2 = filterData(crimes2, polygons).reset_index(drop = True)

# https://www.pittsburghpa.gov/files/assets/city/v/1/public-safety/documents/police/2025_pittsburgh_monthly_incident_dataset_disclaimer.pdf

In [ ]:
# toMerge = nbrhds[['hood', 'lat', 'lng']]
# toMerge.columns = ['nbrhd', 'lat_nbrhd', 'lng_nbrhd']
# crimes2 = (
#     crimes2
#     .merge(right = toMerge, on = 'nbrhd', how = 'left')
#     .assign(
#         lat = lambda x: np.where((x['lat'].notnull()) & (x['lat'] != 0), x['lat'], x['lat_nbrhd']),
#         lng = lambda x: np.where((x['lng'].notnull()) & (x['lng'] != 0), x['lng'], x['lng_nbrhd'])
#     )
#     .dropna(subset = ['lat', 'lng'])
# )

In [73]:
crimes2.to_csv(clean + 'clean_crimes_new.csv', index = False)

In [ ]:
crimes2 = pd.read_csv(clean + 'clean_crimes_new.csv')

### Concatenating crimes

In [74]:
crimes = pd.concat([crimes1, crimes2], axis = 0).reset_index(drop = True)

In [8]:
crimes = crimes.loc[pd.to_datetime(crimes['date'], format = 'mixed').dt.year >= 2020]

In [14]:
crimes.to_csv(clean + 'clean_crimes.csv', index = False)

In [7]:
crimes = pd.read_csv(clean + 'clean_crimes.csv')

### Fires

### Street shapefiles

In [ ]:
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=453e8677-66b0-45c0-8083-b8955ac742c7&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

shapefiles = pd.concat(dfs.values(), ignore_index = True)

In [ ]:
shapefiles.to_csv(raw + 'raw_shapefiles.csv', index = False)

In [ ]:
shapefiles.head(1)

In [ ]:
names = []
types = []
lngs = []
lats = []
for idx, street in shapefiles[['st_name', 'st_type', 'lmuni', 'rmuni']].value_counts().reset_index().iterrows():
    values = shapefiles.loc[
    (shapefiles['st_name'] == street['st_name']) &
    (shapefiles['st_type'] == street['st_type']) &
    (shapefiles['lmuni'] == street['lmuni']) &
    (shapefiles['rmuni'] == street['rmuni']),
    'wkt'].values

    for i in range(len(values)):
        lngs.extend([float(j.strip().split(' ')[0]) for j in values[i][12:-1].split(',')])
        lats.extend([float(j.strip().split(' ')[1]) for j in values[i][12:-1].split(',')])
        n = len([float(j.strip().split(' ')[0]) for j in values[i][12:-1].split(',')])
        names.extend([street['st_name']] * n)
        types.extend([street['st_type']] * n)

In [ ]:
len(names), len(types), len(lats), len(lngs)

In [ ]:
streets = pd.DataFrame({'name': names, 'type': types, 'lat': lats, 'lng': lngs})

In [ ]:
streets = filterData(streets, polygons)

In [ ]:
streets.to_csv(clean + 'street_shapefiles.csv', index = False)

In [ ]:
#streets = pd.read_csv(clean + 'street_shapefiles.csv')

In [ ]:
streets

In [ ]:
# fig = px.scatter_mapbox(streets.iloc[:500], lat = 'lat', lon = 'lng', zoom = 10)
# fig.update_layout(mapbox_style = 'open-street-map')
# fig.show()

### Permits (for demolitions)

In [ ]:
# https://data.wprdc.org/dataset/pli-permits/resource/f4d1177a-f597-4c32-8cbf-7885f56253f6
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=f4d1177a-f597-4c32-8cbf-7885f56253f6&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

permits = pd.concat(dfs.values(), ignore_index = True)

In [ ]:
permits.to_csv(raw + 'raw_permits.csv', index = False)

In [ ]:
permits = pd.read_csv(raw + 'raw_permits.csv')

In [ ]:
permits.head(1)

,_id,permit_id,permit_type,owner_name,contractor_name,work_description,work_type,commercial_or_residential,total_project_value,issue_date,parcel_num,address,latitude,longitude,council_district,neighborhood,ward,zip_code,status
0,3548939,DP-2024-10311,Demolition Permit,EAA HOLDINGS LLC,Lutterman Excavation LLC,DEMO 2 STORY SINGLE FAMILY DETACHED DWELLING,COMPLETE DEMOLITION,Residential,16200.000,2024-09-05,0060L00317000000,No primary address specified,40.398,-79.988,4.000,Carrick,29.000,15210,Completed


In [ ]:
permits = permits[['parcel_num', 'issue_date', 'permit_id', 'permit_type', 'owner_name', 'work_type', 'work_description', 'commercial_or_residential',
                   'total_project_value', 'latitude', 'longitude', 'neighborhood', 'status']]

permits.columns = ['parcelID', 'date', 'permitID', 'permitType', 'ownerName', 'workType', 'desc', 'comm_res', 'value', 'lat', 'lng', 'neighborhood', 'status']

permits['date'] = pd.to_datetime(permits['date'], format = 'mixed')

permits = permits.loc[(permits['date'].dt.year >= 2020) & (permits['permitType'] == 'Demolition Permit')]

In [ ]:
permits.to_csv(clean + 'clean_demos_new.csv', index = False)

In [15]:
demos_new = pd.read_csv(clean + 'clean_demos_new.csv')

In [ ]:
#demos_new.to_csv(clean + 'clean_demos.csv', index = False)

### Historical demolitions preceding 2020 (local CSVs manually downloaded from WPRDC)

Only going back to 2016 because that is as far back as the robust crimes reporting goes, and it is as far back as the code violations goes

In [ ]:
# https://data.wprdc.org/dataset/city-of-pittsburgh-building-permit-summary

folder = Path(raw + 'Historical permits data')

excels = folder.glob('*.xls*')
dfs = []

for i, file in enumerate(excels, start = 1):
    try:
        dfs.append(pd.read_excel(file))
    except:
        print(i, file.name)

In [ ]:
# dict to hold combined dfs per unique column set
combined = {}

for i, df in enumerate(dfs, start=1):
    cols = tuple(df.columns)  # or frozenset(df.columns) if order doesn’t matter
    
    if cols not in combined:
        # start a new combined df for this column set
        combined[cols] = df.copy()
        #print(f"Started new group at DataFrame {i}")
    else:
        # append to the existing combined df
        combined[cols] = pd.concat([combined[cols], df], ignore_index=True)
        #print(f"Appended DataFrame {i} to existing group")

combined = {i: df for i, df in enumerate(combined.values())}

In [ ]:
cols_renames = [
    ['permitID', 'parcelID', 'address', 'ward', 'owner', 'contractor', 'type', 'structure', 'desc', 'value', 'year'],
    ['permitID', 'year', 'parcelID', 'address', 'neighborhood', 'ward', 'owner', 'contractor', 'type', 'structure', 'desc', 'value'],
    ['permitID', 'ward', 'parcelID', 'address', 'owner', 'contractor', 'desc', 'value', 'year'],
    ['permitID', 'year', 'parcelID', 'address', 'neighborhood', 'ward', 'owner', 'contractor', 'type', 'structure', 'desc', 'value'],
    ['permitID', 'year', 'parcelID', 'address', 'neighborhood', 'ward', 'owner', 'contractor', 'type', 'structure', 'desc', 'value'],
    ['year', 'permitID', 'ward', 'parcelID', 'address', 'owner', 'contractor', 'desc', 'value'],
    ['permitID', 'ward', 'parcelID', 'address', 'owner', 'contractor', 'desc', 'value', 'year'],
    ['permitID', 'ward', 'parcelID', 'address', 'owner', 'contractor', 'desc', 'value', 'year']
]

cols_keep = ['permitID', 'parcelID', 'year']
# not all the data has 'type' variable to determine if the demo is partial or complete - will just use all demos including non-complete ones

for i, names in zip(combined, cols_renames):
    combined[i].columns = names
    combined[i] = combined[i].dropna().loc[lambda x: x['permitID'].str.contains('D')]#, cols_keep]

# demos = pd.concat([combined[i] for i in combined], axis = 0).reset_index(drop = True)
# demos['parcelID'] = demos['parcelID'].str.replace('-', '')
# demos = demos.loc[demos['parcelID'].str.len() == 16]
# demos['year'] = pd.to_datetime(demos['year'])
# demos.loc[demos['year'].astype(str).str[-4:] != '0000', 'year'] = demos['year'].astype(str).str[-4:]
# demos['year'] = demos['year'].dt.year

In [ ]:
demos_new['status'].value_counts(normalize = True)

status
Completed                          0.755
Issued                             0.121
Expired                            0.114
Revoked                            0.003
Application Finalization           0.003
Amendment Application Incomplete   0.002
Amendment Applicant Revisions      0.001
Name: proportion, dtype: float64

In [ ]:
combined[0]['desc'].value_counts(normalize = True)

desc
RAZE GARAGE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            0.049
DEMOLISH STRUCTURE, CAP UTILITIES, BACKFILL AND GRADE                                                                                                                                                                                                                                                                                                                 

In [ ]:
demos.to_csv(clean + 'clean_demos_old.csv', index = False)

In [ ]:
demos_old = pd.read_csv(clean + 'clean_demos_old.csv')

In [ ]:
demos_new['year'] = pd.to_datetime(demos_new['date'], format = 'mixed').dt.year
demos_new = demos_new[['permitID', 'parcelID', 'year']]

In [ ]:
demos = pd.concat([demos_new, demos_old], axis = 0).sort_values(by = 'year').reset_index(drop = True)

In [ ]:
demos = demos.merge(right = parcels[['parcelID', 'lat', 'lng', 'nbrhd']], on = 'parcelID', how = 'inner')

In [ ]:
demos.to_csv(clean + 'clean_demos.csv', index = False)

In [ ]:
demos = pd.read_csv(clean + 'clean_demos.csv')

### Condemnations

In [ ]:
# https://data.wprdc.org/dataset/condemned-properties/resource/0a963f26-eb4b-4325-bbbc-3ddf6a871410
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/action/datastore_search?resource_id=0a963f26-eb4b-4325-bbbc-3ddf6a871410&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

conds = pd.concat(dfs.values(), ignore_index = True)

In [ ]:
parcels = pd.read_csv(clean + 'blight_and_values.csv')

In [ ]:
conds = conds.rename(columns = {'parcel_id': 'parcelID', 'property_type': 'status'})

In [ ]:
merge = (
    parcels
    .loc[parcels['year'] >= 2020]
    .merge(right = conds.assign(year = pd.to_datetime(conds['date']).dt.year)[['parcelID', 'year', 'status']], on = ['parcelID', 'year'], how = 'left')
    .assign(
       status = lambda x: np.where(~x['status'].isna(), x['status'], 'null')
    )
)

In [ ]:
merge.groupby(['year', 'status'])['viols_count_decayed'].mean()

year  status            
2020  Condemned Property   1169.028
      Dead End Property    1372.790
      null                  830.593
2021  Condemned Property   1092.503
      Dead End Property    1038.073
      null                  834.679
2022  Condemned Property   1145.119
      Dead End Property    1122.538
      null                  834.410
2023  Condemned Property   1104.653
      Dead End Property    1169.347
      null                  834.148
2024  Condemned Property    965.906
      Dead End Property    1165.831
      null                  834.318
2025  Condemned Property   1010.821
      Dead End Property    1129.472
      null                  834.854
Name: viols_count_decayed, dtype: float64

In [ ]:
for y in [2020, 2021, 2022, 2023, 2024, 2025]:
    subset = merge.loc[merge['year'] == y]
    groups = [subset.loc[subset['status'] == s]['viols_count_decayed'] for s in subset['status'].unique()]

    f_stat, p_val = f_oneway(*groups)
    print(f'Year {y}: F = {f_stat:.3f}, p = {p_val:.3f}')

Year 2020: F = 356.268, p = 0.000
Year 2021: F = 28.894, p = 0.000
Year 2022: F = 52.830, p = 0.000
Year 2023: F = 71.316, p = 0.000
Year 2024: F = 52.965, p = 0.000
Year 2025: F = 21.591, p = 0.000


In [ ]:
merge.groupby(['year', 'status'])['low_count_decayed'].mean()

year  status            
2020  Condemned Property   861.442
      Dead End Property    927.688
      null                 688.487
2021  Condemned Property   845.390
      Dead End Property    747.517
      null                 690.576
2022  Condemned Property   930.423
      Dead End Property    805.653
      null                 690.447
2023  Condemned Property   898.606
      Dead End Property    733.294
      null                 690.418
2024  Condemned Property   703.961
      Dead End Property    816.294
      null                 690.574
2025  Condemned Property   770.683
      Dead End Property    743.191
      null                 690.755
Name: low_count_decayed, dtype: float64

In [ ]:
for y in [2020, 2021, 2022, 2023, 2024, 2025]:
    subset = merge.loc[merge['year'] == y]
    groups = [subset.loc[subset['status'] == s]['low_count_decayed'] for s in subset['status'].unique()]

    f_stat, p_val = f_oneway(*groups)
    print(f'Year {y}: F = {f_stat:.3f}, p = {p_val:.3f}')

Year 2020: F = 90.157, p = 0.000
Year 2021: F = 8.888, p = 0.000
Year 2022: F = 17.234, p = 0.000
Year 2023: F = 17.100, p = 0.000
Year 2024: F = 6.942, p = 0.001
Year 2025: F = 1.618, p = 0.198


##### Commentary
This demonstrates that violations are a predictor of condemned and dead-end properties. Along with the understanding that "vacant and abandoned buildings in urban neighborhoods are themselves a manifestation of social disorganization," (Wheeler et al 2018 - The Effect of Housing Demolitions on Crime in Buffalo, New York), this supports the idea that code violations are indicative of social disorganization and can thus be used as a proxy for urban blight. Doing the ANOVA on crime also suggests a relationship between condemnations and crime.

### Property Sales

In [ ]:
# https://data.wprdc.org/dataset/real-estate-sales/resource/5bbe6c55-bce6-4edb-9d04-68edeb6bf7b1
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=5bbe6c55-bce6-4edb-9d04-68edeb6bf7b1&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

sales = pd.concat(dfs.values(), ignore_index = True)

In [ ]:
sales.shape

In [ ]:
sales.head(1)

In [ ]:
sales.to_csv(raw + 'raw_sales.csv', index = False)

In [7]:
sales = pd.read_csv(raw + 'raw_sales.csv')

In [8]:
sales = sales[['PARID', 'SALEDATE', 'PRICE']]

sales.columns = ['parcelID', 'date', 'value']

### Appeals 1

In [ ]:
# https://data.wprdc.org/api/3/action/datastore_search?resource_id=8eff881d-4d28-4064-83f1-30cc991cfec7
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=65855e14-549e-4992-b5be-d629afc676fa&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

appeals1 = pd.concat(dfs.values(), ignore_index = True)

In [ ]:
appeals1.shape

In [ ]:
appeals1.head(1)

,_id,PARID,PROPERTYHOUSENUM,PROPERTYFRACTION,PROPERTYADDRESS,PROPERTYCITY,PROPERTYSTATE,PROPERTYUNIT,PROPERTYZIP,MUNICODE,MUNIDESC,SCHOOLCODE,SCHOOLDESC,LEGAL1,LEGAL2,LEGAL3,NEIGHCODE,NEIGHDESC,TAXCODE,TAXDESC,TAXSUBCODE,TAXSUBCODE_DESC,OWNERCODE,OWNERDESC,CLASS,CLASSDESC,USECODE,USEDESC,LOTAREA,HOMESTEADFLAG,CLEANGREEN,FARMSTEADFLAG,ABATEMENTFLAG,RECORDDATE,SALEDATE,SALEPRICE,SALECODE,SALEDESC,DEEDBOOK,DEEDPAGE,PREVSALEDATE,PREVSALEPRICE,PREVSALEDATE2,PREVSALEPRICE2,CHANGENOTICEADDRESS1,CHANGENOTICEADDRESS2,CHANGENOTICEADDRESS3,CHANGENOTICEADDRESS4,COUNTYBUILDING,COUNTYLAND,COUNTYTOTAL,COUNTYEXEMPTBLDG,LOCALBUILDING,LOCALLAND,LOCALTOTAL,FAIRMARKETBUILDING,FAIRMARKETLAND,FAIRMARKETTOTAL,STYLE,STYLEDESC,STORIES,YEARBLT,EXTERIORFINISH,EXTFINISH_DESC,ROOF,ROOFDESC,BASEMENT,BASEMENTDESC,GRADE,GRADEDESC,CONDITION,CONDITIONDESC,CDU,CDUDESC,TOTALROOMS,BEDROOMS,FULLBATHS,HALFBATHS,HEATINGCOOLING,HEATINGCOOLINGDESC,FIREPLACES,BSMTGARAGE,FINISHEDLIVINGAREA,CARDNUMBER,ALT_ID,TAXYEAR,ASOFDATE
0,14151259,0001G00106000000,0.000,,MARKET ST,PITTSBURGH,PA,,15222.000,101,1st Ward - PITTSBURGH,47,Pittsburgh,LOT 38.67X60.19,PT EXTINGUISHED ALLEY = 21.75X4X21.75X4,NaN,51C02,PITTSBURGH URBAN,T,20 - Taxable,NaN,NaN,20,CORPORATION,C,COMMERCIAL,400,VACANT COMMERCIAL LAND,2415,NaN,NaN,NaN,NaN,12-30-2016,11-11-2016,10.000,H,MULTI-PARCEL SA,16656,25.000,12-18-1991,210000.000,NaN,NaN,2020 SMALLMAN ST STE 301,,PITTSBURGH PA,15222.000,0,61400,61400,0,0,61400,61400,0,61400,61400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,2025-08-01


In [ ]:
appeals1.to_csv(raw + 'appeals1.csv', index = False)

In [9]:
appeals1 = pd.read_csv(raw + 'appeals1.csv')

In [10]:
appeals1 = appeals1[['PARID', 'SALEDATE', 'SALEPRICE', 'PREVSALEDATE', 'PREVSALEPRICE', 'PREVSALEDATE2', 'PREVSALEPRICE2']]
#appeals1[['PARID', 'ASOFDATE', 'FAIRMARKETTOTAL']]

appeals1.columns = ['parcelID', 'date0', 'value0', 'date1', 'value1', 'date2', 'value2'] # '0' most recent

In [11]:
temp = pd.DataFrame()
for i in range(3):
    slice = appeals1[['parcelID', f'date{i}', f'value{i}']]
    slice.columns = ['parcelID', 'date', 'value']
    temp = pd.concat([temp, slice], axis = 0)
appeals1 = temp.copy()

### Appeals 2

In [ ]:
# https://data.wprdc.org/dataset/property-data-with-geographic-identifiers/resource/8eff881d-4d28-4064-83f1-30cc991cfec7
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=8eff881d-4d28-4064-83f1-30cc991cfec7&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

appeals2 = pd.concat(dfs.values(), ignore_index = True)

In [ ]:
appeals2.shape

In [ ]:
appeals2.head(1)

,_id,PARID,PROPERTYHOUSENUM,PROPERTYFRACTION,PROPERTYADDRESS,PROPERTYCITY,PROPERTYSTATE,PROPERTYUNIT,PROPERTYZIP,MUNICODE,MUNIDESC,SCHOOLCODE,SCHOOLDESC,LEGAL1,LEGAL2,LEGAL3,NEIGHCODE,NEIGHDESC,TAXCODE,TAXDESC,TAXSUBCODE,TAXSUBCODE_DESC,OWNERCODE,OWNERDESC,CLASS,CLASSDESC,USECODE,USEDESC,LOTAREA,HOMESTEADFLAG,CLEANGREEN,FARMSTEADFLAG,ABATEMENTFLAG,RECORDDATE,SALEDATE,SALEPRICE,SALECODE,SALEDESC,DEEDBOOK,DEEDPAGE,PREVSALEDATE,PREVSALEPRICE,PREVSALEDATE2,PREVSALEPRICE2,CHANGENOTICEADDRESS1,CHANGENOTICEADDRESS2,CHANGENOTICEADDRESS3,CHANGENOTICEADDRESS4,COUNTYBUILDING,COUNTYLAND,COUNTYTOTAL,COUNTYEXEMPTBLDG,LOCALBUILDING,LOCALLAND,LOCALTOTAL,FAIRMARKETBUILDING,FAIRMARKETLAND,FAIRMARKETTOTAL,STYLE,STYLEDESC,STORIES,YEARBLT,EXTERIORFINISH,EXTFINISH_DESC,ROOF,ROOFDESC,BASEMENT,BASEMENTDESC,GRADE,GRADEDESC,CONDITION,CONDITIONDESC,CDU,CDUDESC,TOTALROOMS,BEDROOMS,FULLBATHS,HALFBATHS,HEATINGCOOLING,HEATINGCOOLINGDESC,FIREPLACES,BSMTGARAGE,FINISHEDLIVINGAREA,CARDNUMBER,ALT_ID,TAXYEAR,ASOFDATE,MUNICIPALITY,NEIGHBORHOOD,PGH_COUNCIL_DISTRICT,PGH_WARD,PGH_PUBLIC_WORKS_DIVISION,PGH_POLICE_ZONE,PGH_FIRE_ZONE,TRACT,BLOCK_GROUP
0,1,0001G00224060300,151.000,,FORT PITT BLVD,PITTSBURGH,PA,UNIT 603,15222,101,1st Ward - PITTSBURGH,47,Pittsburgh,FIRST SIDE CONDOMINIUM - AMENDMENT OF DECLARAT...,UNIT 603 & PARKING SPACE,6TH LEVEL,61P03H,151 FIRST,T,20 - Taxable,NaN,NaN,12,REGULAR-ETUX OR ET VIR,R,RESIDENTIAL,50,CONDOMINIUM,0,HOM,NaN,NaN,NaN,10-05-2017,09-29-2017,699000.000,36,QUIT CLAIM / SPEC WARRNTY,16966,186.000,10-25-2013,460000.000,08-23-2007,389745.000,151 FORT PITT BLVD UNIT 603,,PITTSBURGH PA,15222.000,592900,0,592900,0,610900,0,610900,610900,0,610900,21.000,CONDO HR,1.000,2007.000,7.000,Concrete,4.000,ROLL,1.000,NaN,A+,VERY GOOD +,3.000,AVERAGE,AV,AVERAGE,5.000,2.000,2.000,0.000,B,Central Heat with AC,NaN,NaN,1761.000,1.000,NaN,2018,2018-09-01,Pittsburgh,Central Business District,6.000,1.000,6.000,2.000,1-4,42003020100.000,420030201001.000


In [ ]:
appeals2.to_csv(raw + 'appeals2.csv', index = False)

In [12]:
appeals2 = pd.read_csv(raw + 'appeals2.csv')

In [13]:
appeals2 = appeals2[['PARID', 'SALEDATE', 'SALEPRICE', 'PREVSALEDATE', 'PREVSALEPRICE', 'PREVSALEDATE2', 'PREVSALEPRICE2']]
#appeals2[['PARID', 'ASOFDATE', 'FAIRMARKETTOTAL']]

appeals2.columns = ['parcelID', 'date0', 'value0', 'date1', 'value1', 'date2', 'value2'] # '0' most recent

In [14]:
temp = pd.DataFrame()
for i in range(3):
    slice = appeals2[['parcelID', f'date{i}', f'value{i}']]
    slice.columns = ['parcelID', 'date', 'value']
    temp = pd.concat([temp, slice], axis = 0)
appeals2 = temp.copy()

### Appeals 3

In [ ]:
# https://data.wprdc.org/dataset/allegheny-county-property-assessment-appeals/resource/8a7607fb-c93e-4d7a-9b23-528b5c25b1de
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=8a7607fb-c93e-4d7a-9b23-528b5c25b1de&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

appeals3 = pd.concat(dfs.values(), ignore_index = True)

In [ ]:
appeals3.shape

In [ ]:
appeals3.head(1)

In [ ]:
appeals3.to_csv(raw + 'appeals3.csv', index = False)

In [15]:
appeals3 = pd.read_csv(raw + 'appeals3.csv')

In [16]:
appeals3 = appeals3[['PARCEL ID', 'DISPO DATE', 'CURRENT TOTAL VALUE']]

appeals3.columns = ['parcelID', 'date', 'value']

### Concatenating values (all appeals + sales)

In [ ]:
sales.head(1)

In [ ]:
appeals1.head(1)

In [ ]:
appeals2.head(1)

In [ ]:
appeals3.head(1)

In [17]:
values = pd.concat([appeals1, appeals2, appeals3, sales], axis = 0)
values['date'] = pd.to_datetime(values['date'], format = 'mixed')

In [18]:
# merging in lat, lng - also filtering for Pittsburgh parcels
values = values.merge(right = parcels, on = 'parcelID', how = 'right')

# sort by date ascending
values = values.sort_values(by = 'date')

# filter dates including 2016 to be able to do smoothing for 2020
values = values.loc[(values['date'].dt.year >= 2016) & (values['date'].dt.year <= 2025)]

# removing abnormally large values (x > $10mm) to smooth results
values = values.loc[values['value'] <= 10_000_000]

# remove year-by-year duplicates
values = (
    values
    .assign(year = values['date'].dt.year)
    .drop_duplicates(subset = ['parcelID', 'year'], keep = 'last')
)

# drop nulls
values = values.dropna()

# reset the final index
values = values.reset_index(drop = True)

In [19]:
values.to_csv(clean + 'clean_values.csv', index = False)

In [17]:
values = pd.read_csv(clean + 'clean_values.csv')